In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import rpy2.robjects as robjects
import rpy2.robjects.pandas2ri as pandas2ri
from rpy2.robjects.conversion import localconverter
from rpy2.robjects.packages import importr, isinstalled
import rpy2.robjects.packages as rpackages
from rpy2.robjects import r
import sys

import sys
sys.path.insert(0, '/Users/alanma/Documents/CFA_python')
import faircause.faircause as faircause


In [2]:
# Ensure faircause is installed (either using conda or R)

print("Is faircause installed now?", isinstalled('faircause'))


Is faircause installed now? True


In [3]:
# import
base = importr('base')
faircause = importr('faircause')

# Load census dataset
data = robjects.r('''
    data("gov_census", package = "faircause")
    gov_census[seq_len(20000), ]  # Take first 20000 rows as in the vignette
''')

# Rename columns to match the vignette
data.columns = ['sex', 'age', 'race', 'hispanic_origin', 'citizenship', 'nativity',
                'marital', 'family_size', 'children', 'education_level', 'english_level',
                'salary', 'hours_worked', 'weeks_worked', 'occupation', 'industry',
                'economic_region']

# Convert to pd
with localconverter(robjects.default_converter + pandas2ri.converter):
  data = robjects.conversion.rpy2py(data)
data.reset_index(drop=True, inplace=True)


    an issue that caused a segfault when used with rpy2:
    https://github.com/rstudio/reticulate/pull/1188
    Make sure that you use a version of that package that includes
    the fix.
    

In [4]:
data

,sex,age,race,hispanic_origin,citizenship,nativity,marital,family_size,children,education_level,english_level,salary,hours_worked,weeks_worked,occupation,industry,economic_region
0,male,64.0,black,no,1.0,native,married,2.0,0.0,20.0,0.0,43000.0,56.0,49.0,13-1081,928P,Southeast
1,female,54.0,white,no,1.0,native,married,3.0,1.0,20.0,0.0,45000.0,42.0,49.0,29-2061,6231,Southeast
2,male,38.0,black,no,1.0,native,married,3.0,1.0,24.0,0.0,99000.0,50.0,49.0,25-1000,611M1,Southeast
3,female,41.0,asian,no,1.0,native,married,3.0,1.0,24.0,0.0,63000.0,50.0,49.0,25-1000,611M1,Southeast
4,female,40.0,white,no,1.0,native,married,4.0,2.0,21.0,0.0,45200.0,40.0,49.0,27-1010,611M1,Southeast
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19995,female,54.0,white,no,1.0,native,married,2.0,0.0,22.0,0.0,38000.0,40.0,39.0,27-3092,923,Far West
19996,male,43.0,other,yes,4.0,foreign-born,divorced,4.0,0.0,21.0,1.0,100000.0,40.0,49.0,33-3021,92MP,Far West
19997,male,35.0,AIAN,yes,4.0,foreign-born,married,4.0,2.0,22.0,2.0,73000.0,40.0,47.0,25-2030,6111,Far West
19998,male,26.0,white,no,1.0,native,married,2.0,0.0,18.0,0.0,60000.0,75.0,49.0,55-2010,928110P4,Far West


In [5]:
# placing columns
X = "sex"
W = ["marital", "family_size", "children", "education_level",
    "english_level", "hours_worked", "weeks_worked", "occupation",
    "industry"]
Z = ["age", "race", "hispanic_origin", "citizenship", "nativity",
    "economic_region"]
Y = "salary"
x0 = "male"
x1 = "female"


In [6]:
import faircause.faircause as faircause

fc_census = faircause.FairCause(data, X, Z, W, Y, x0, x1)

In [7]:
print(fc_census)

faircause object:

Attribute:       sex
Outcome:         salary
Confounders:     marital, family_size, children, education_level, english_level, hours_worked, weeks_worked, occupation, industry
Mediators:       age, race, hispanic_origin, citizenship, nativity, economic_region


In [8]:
fc_census.estimate_effects()

64816.60112969802 -49027.1271076524
64815.702524103566 -50451.467174296595
65172.33742463394 -49316.03808812547
64785.03050773589 -49448.75993346886
64113.39565546309 -49526.982413696845
63925.44487847222 -49866.155415430265
64854.40321715818 -49467.79391100703
64399.50988822012 -49903.59386686612
64993.098470157296 -49439.64545624184
63671.45835595613 -49796.06245945695
65236.07162176783 -48999.11778376871
65171.33146309883 -50140.64386880358
64869.73777921657 -50150.19938507407
65111.498364231185 -49432.324099722995
65373.192299325645 -50392.29502128447
64285.19234925438 -50420.946398659966
65149.0413330448 -49601.52072875999
65133.262587980505 -49783.557826288896
65125.90045841519 -49486.43661192102
64220.373628760724 -49577.232465486886
65336.353144213135 -49345.63239817318
63946.448984026945 -49705.66546262851
65871.42202323845 -50389.30407759676
63626.538951390365 -50034.73624503687
66223.33260940385 -49870.18255954036
65654.65303926842 -49866.89023820645
65028.47578499202 -49759

/Users/alanma/Documents/CFA_python/faircause/estimation/mediation_dml.py:155: RuntimeWarning: divide by zero encountered in divide
  y0.append((Y.loc[ts].values - y_z0_ts) * (X.loc[ts] == 0).values / (1-px_z_ts) + y_z0_ts)
/Users/alanma/Documents/CFA_python/faircause/estimation/mediation_dml.py:155: RuntimeWarning: invalid value encountered in divide
  y0.append((Y.loc[ts].values - y_z0_ts) * (X.loc[ts] == 0).values / (1-px_z_ts) + y_z0_ts)
/Users/alanma/Documents/CFA_python/faircause/estimation/mediation_dml.py:156: RuntimeWarning: divide by zero encountered in divide
  y1.append((Y.loc[ts].values - y_z1_ts) * (X.loc[ts] == 1).values / (px_z_ts) + y_z1_ts)
/Users/alanma/Documents/CFA_python/faircause/estimation/mediation_dml.py:156: RuntimeWarning: invalid value encountered in divide
  y1.append((Y.loc[ts].values - y_z1_ts) * (X.loc[ts] == 1).values / (px_z_ts) + y_z1_ts)
/Users/alanma/Documents/CFA_python/faircause/estimation/mediation_dml.py:155: RuntimeWarning: divide by zero encou

3.98 percent of extreme P(x|z) or p(x|z,w) prob
 Reported results are for the overlap pop. Consider investigating overlap issues
47431.75692992324 -64201.940970262694
48312.04160600779 -63385.196136142105
50273.88186432682 -64735.36876379475
49248.62772794777 -64719.419133006086
49219.820949828856 -62845.81706141888
49456.76360887535 -64171.01830810739
49580.56433139114 -65660.06373734132
48460.10954197296 -66920.65747141253
49360.47132632311 -65508.79932456974
48807.683790543946 -65473.750224533316
50291.47936312163 -62345.8047492204
49321.595320569715 -63467.05227827719
49494.724464915045 -64056.517332002826
50020.46954006168 -65585.40978561706
49803.702530682276 -63856.787944158656
48545.58334258842 -63358.08978126463
49114.854909700065 -64411.251750955365
48769.78335510736 -64754.0198289782
49484.47061123978 -62762.51555505243
48424.94321264789 -63279.304303483535
47966.040723048514 -63937.98007956147
49343.98268281792 -64073.51244927136
49874.008991490875 -64007.88630114596
49186.

In [9]:
fc_census.summary()

faircause object summary:

Protected attribute:                 sex
Protected attribute levels:          male, female
Total Variation (TV): 15086.5287

TV decomposition(s):

TV_malefemale(y) (15086.5287) = CtfDE_malefemale(y | male) (11720.0800) - CtfIE_femalemale(y | male) (11573.8563) - CtfSE_femalemale(y) (-14940.3050)


,measure,value,sd
0,ctfde,11720.079998,3778.528772
1,ctfie,11573.856272,3648.155373
2,ctfse,-14940.304967,982.351058
3,ett,146.223727,1009.464362
4,expse_x0,-15042.964958,1135.765228
5,expse_x1,15597.980476,790.312595
6,nde,-4495.557749,2353.100018
7,nie,11058.858992,2237.719050
8,te,-15554.416740,1121.040016
9,tv,15086.528694,733.334221


In [10]:
print(np.nanmean(data[Y][data[X] == x0]) -  np.nanmean(data[Y][data[X] == x1]))

15053.691747379096
